# DATA EXPLORATION

In [ ]:
import os 
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import torchvision
import random
from sklearn.model_selection import train_test_split

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
os.listdir(path)

In [ ]:
path

In [ ]:
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
train

VAMOS A MIRAR SI EL DATASET ESTÁ BALANCEADO

In [ ]:
train.label.hist()
plt.show()

In [ ]:
train.label.value_counts()

CLARAMENTE DESBALANCEADO. EN PRIMERA INSTANCIA SE PROCEDERÁ A TIRAR TAL CUAL. SI NO CARRULA YA VEREMOS

In [ ]:
r, c = 3, 5
fig = plt.figure(figsize=(3*c, 3*r))
for _r in range(r):
    for _c in range(c):
        ax = plt.subplot(r, c, _r*c + _c + 1)
        ix = random.randint(0, len(train)-1)
        img = torchvision.io.read_image(str(path/'train_images'/train['image_id'][ix]))
        label = train['label'][ix]
        plt.axis("off")
        plt.imshow(img.permute(1,2,0))
        plt.title(label)
plt.tight_layout()
plt.show()

In [ ]:
img.shape, img.dtype, img.max(), img.min()

AHORA INSTALAMOS Y CARGAMOS __PYTORCH LIGHTNING__

In [ ]:
!pip install pytorch-lightning==1.1.8

In [ ]:
!pip install torch==1.8.1

In [ ]:
import torch
import pytorch_lightning  as pl
from torch.utils.data import DataLoader

In [ ]:
# creo primero la clase Dataset

class Dataset(torch.utils.data.Dataset):
    
    def __init__(self,imgs,labels):
        self.imgs = imgs
        self.labels = labels
        
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self,ix):
        img = torchvision.io.read_image(self.imgs[ix]).float()/255.
        
        label = torch.tensor(self.labels[ix], dtype=torch.long)
        return img, label
    
    
    

class DataModule(pl.LightningDataModule):

    def __init__(self, 
                 path=Path('../input/cassava-leaf-disease-classification'), 
                 batch_size = 64, 
                 test_size = 0.2, 
                 random_state=42, 
                 subset = 0
                 ):
        super().__init__()
        self.path = path
        self.batch_size = batch_size
        self.test_size = test_size
        self.random_state = random_state
        self.subset = subset

    def setup(self, stage=None):
        #dataframe
        df= pd.read_csv(self.path/'train.csv')
        #separar en train y validation
        train, val =train_test_split(
            df,
            test_size=self.test_size,
            shuffle=True,
            stratify= df['label'],
            random_state = self.random_state
        )
        print('Training samples: ' , len(train))
        print('Validation samples: ', len(val))
        if self.subset:
            _, subset = train_test_split(
                train,
                test_size = self.subset,
                shuffle=True,
                stratify = train['label'],
                random_state = self.random_state
            )
            
            print('Training with ', len(subset), 'samples.')
            train_imgs = [str(self.path/'train_images'/img) for img in subset['image_id'].values]
            train_labels = subset['label'].values
        else:
            train_imgs = [str(self.path/'train_images'/img) for img in train['image_id'].values]
            train_labels = train['label'].values
            
            
        #definimos los dos dataset
        self.train_ds = Dataset(train_imgs, train_labels)
        
        val_imgs = [str(self.path/'train_images'/img) for img in val['image_id'].values]
        val_labels = val['label'].values
        self.val_ds = Dataset(val_imgs, val_labels)
            
    def train_dataloader(self):
        return DataLoader(self.train_ds, batch_size=self.batch_size, shuffle = True) 

    def val_dataloader(self):
        return DataLoader(self.val_ds, batch_size=self.batch_size, shuffle = False)


In [ ]:
 trans = torchvision.transforms.Compose([
        torchvision.transforms.Resize((32,32))
 ])

In [ ]:
dm = DataModule(subset=0)
dm.setup()

In [ ]:
r, c = 3, 5
fig = plt.figure(figsize=(3*c, 3*r))
ds = dm.train_ds
for _r in range(r):
    for _c in range(c):
        ax = plt.subplot(r, c, _r*c + _c + 1)
        ix = random.randint(0, len(ds)-1)
        img, lablel = ds[ix]
        label = train['label'][ix]

        
        plt.axis("off")
        plt.imshow(img.permute(1,2,0))
        plt.title(label.item())
plt.tight_layout()
plt.show()

In [ ]:
img.shape, img.dtype, img.max(), img.min()

En principio este DataModule ya estaría, imagenes float, en tensor y normalizadas.
Vamos a comprobar los dataloaders.

In [ ]:
#DataLoader
dl=dm.train_dataloader()

#Batch de imagenes y etiquetas
imgs, labels = next(iter(dl))

#64 imagenes
imgs.shape, imgs.dtype, imgs.max(), imgs.min()

Modelo de prueba, Resnet

In [ ]:
import torch.nn.functional as F
from pytorch_lightning.metrics.functional import accuracy


In [ ]:
m_plus = 0.9
m_minus = 0.1
lambda_ = 0.5
def margin_loss(y,caps2_output):
    T = tf.one_hot(y, depth=caps2_n_caps, name="T").numpy()
    T=torch.from_numpy(T)
    caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True)

    
    present_error_raw = torch.square(torch.max(torch.zeros(1), m_plus - caps2_output_norm))
    present_error = torch.reshape(present_error_raw, shape=(-1, 4))

    absent_error_raw = torch.square(torch.max(torch.zeros(1), caps2_output_norm - m_minus))
    absent_error = torch.reshape(absent_error_raw, shape=(-1, 4))


    L = torch.add(T * present_error, lambda_ * (1.0 - T) * absent_error)

    margin_loss = torch.mean(torch.sum(L, 1))
    return(margin_loss)

In [ ]:
class Model(pl.LightningModule):

    def __init__(self, config, n_classes=5):
        super().__init__()
        self.save_hyperparameters(config)
        self.resnet = torchvision.models.resnet18(pretrained=True)
        self.resnet.fc = torch.nn.Linear(self.resnet.fc.in_features, n_classes)
        #se transforman aqui las imagenes
        self.trans = torch.nn.Sequential(
            torchvision.transforms.CenterCrop(self.hparams.size)
        )

    def forward(self, x):
        # entra batch de imagenes
        return self.resnet(x)

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        x = self.trans(x)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)
        # Logging to TensorBoard by default
        self.log('loss',loss)
        self.log('acc', acc, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)
        # Logging to TensorBoard by default
        self.log('val_loss',loss,prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer

In [ ]:
config = {'lr':3e-4,
          'batch_size':32,
          'precision':16,
          'epochs':3,
          'subset': 0.1,
          'size':256
         }

dm = DataModule(batch_size=config['batch_size'], subset=config['subset'])

model = Model(config)

trainer = pl.Trainer(gpus=1,
    precision=config['precision'],
    max_epochs=config['epochs'],
    limit_train_batches = 5,
    limit_val_batches = 5
    )


trainer.fit(model,dm)


#CAPS NET

In [ ]:
import torch.nn as nn
#import pytorch

In [ ]:
caps2_n_caps = 4
caps2_n_dims = 16

caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules
caps1_n_dims = 8
batch_size=1

In [ ]:
def squash(s,axis=-1,epsilon=1e-5):
    sn=torch.sum(s**2,dim=axis,keepdim=True)
    v=sn/(1+sn)*s/torch.sqrt(sn+epsilon)
    return v

In [ ]:
def safe_norm(s,epsilon=1e-5):
    sn=torch.sum(s**2,dim=-2,keepdim=True)
    v=torch.sqrt(sn+epsilon)
    return v

In [ ]:
class PrimaryCapsules(nn.Module):
    def __init__(self,batch_size):
        super(PrimaryCapsules,self).__init__()
        self.conv1=nn.Conv2d(1,256,(9,9))
        self.conv2=nn.Conv2d(256,256,(9,9),2)
        self.batch_size=batch_size
        
    def forward(self,x):
        out=F.relu(self.conv1(x))
        out=F.relu(self.conv2(out))
        out=out.reshape([self.batch_size,32*6*6,8])
        out=squash(out)
        
        return out

In [ ]:
class DigitCapsule(nn.Module):
    def __init__(self,batch_size,caps1_n_dims,caps2_n_dims,caps2_n_caps,caps1_n_caps,routings=4):
        super(DigitCapsule,self).__init__()
        self.caps1_n_caps=caps1_n_caps
        self.caps2_n_caps=caps2_n_caps
        self.caps2_n_dims=caps2_n_dims
        self.caps1_n_dims=caps1_n_dims
        self.batch_size=batch_size
        self.W = torch.nn.Parameter(torch.normal(0, 0.1, size=(1, self.caps1_n_caps, self.caps2_n_caps, self.caps2_n_dims, self.caps1_n_dims)))
        self.routings=routings
        
    def forward(self,x):
        W=self.W.repeat(self.batch_size,1,1,1,1)
        W=W
        x=x.unsqueeze(dim=-1)
        x=x.unsqueeze(dim=2)
        x=x.repeat([1,1,10,1,1])
        x_predicted=torch.matmul(W,x)
        #Cuidao aqui, puede estar mal
        b = torch.zeros([self.batch_size, self.caps1_n_caps, self.caps2_n_caps, 1, 1]).to(device=device)
        for i in range(self.routings):
            b_weights =F.softmax(b,dim=2)
            if i == self.routings - 1:
                outputs = squash(torch.sum(torch.mul(b_weights, x_predicted),1, keepdim=True),-2)
            else:
                outputs = squash(torch.sum(torch.mul(b_weights, x_predicted),1, keepdim=True),-2)                                 
                b = b + torch.matmul(torch.transpose(x_predicted,3,4), outputs.repeat(1, self.caps1_n_caps, 1, 1, 1))

        return(safe_norm(outputs))

In [ ]:
class CapsuleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.PC=PrimaryCapsules(batch_size=10)
        self.DC=DigitCapsule(batch_size=10,caps1_n_caps=1152,caps1_n_dims=8,caps2_n_caps=10,caps2_n_dims=16)
    
    def forward(self,x):
        x=self.PC(x)
        x=self.DC(x)
        #x=F.softmax(x,dim=2)
        x=x.squeeze(1)
        x=x.squeeze(2)
        return x

In [ ]:
class CapsuleNet(pl.LightningModule):

    def __init__(self,config):
        super().__init__()
        self.save_hyperparameters(config)
        self.PC=PrimaryCapsules(batch_size=10)
        self.DC=DigitCapsule(batch_size=10,caps1_n_caps=1152,caps1_n_dims=8,caps2_n_caps=10,caps2_n_dims=16)

    def forward(self,x):
        x=self.PC(x)
        x=self.DC(x)
        #x=F.softmax(x,dim=2)
        x=x.squeeze(1)
        x=x.squeeze(2)
        return x

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        #x = self.trans(x)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        #loss = margin_loss(y,y_hat)
        acc = accuracy(y_hat, y)
        # Logging to TensorBoard by default
        self.log('loss',loss)
        self.log('acc', acc, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)
        # Logging to TensorBoard by default
        self.log('val_loss',loss,prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer

In [ ]:
config_2 = {'lr':3e-4,
          'batch_size':32,
          'precision':16,
          'epochs':3,
          'subset': 0.1,
          'size':256
         }

dm = DataModule(batch_size=config_2['batch_size'], subset=config_2['subset'])

model_2 = CapsuleNet(config_2)

trainer = pl.Trainer(gpus=1,
    precision=config_2['precision'],
    max_epochs=config_2['epochs'],
    limit_train_batches = 5,
    limit_val_batches = 5
    )


trainer.fit(model_2,dm)

# SEGUNDA VERSIÓN

In [ ]:
caps2_n_caps = 4
caps2_n_dims = 16

caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules
caps1_n_dims = 8
batch_size=1

In [ ]:
def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False):
    squared_norm = torch.sum(s**2,axis, keepdim=keep_dims)
    return torch.sqrt(squared_norm + epsilon)

In [ ]:
def squash(s, axis=-1, epsilon=1e-7):    
    squared_norm = torch.sum(s**2,axis, keepdim=True)
    safe_norm = torch.sqrt(squared_norm + epsilon)
    squash_factor = squared_norm / (1. + squared_norm)
    unit_vector = s / safe_norm
    return squash_factor * unit_vector

In [ ]:
class CapsuleLayer(torch.nn.Module):
    #Capa capsulada
    #caps1_n_caps : numero de capsulas
    #caps1_n_dims: dimension de las capsulas de entrada
    #caps2_n_caps: numero de capsulas de salida de la capa
    #caps2_n_dims: dimension de las capsulas de salida
    #routings: iteraciones del routing
    def __init__(self, caps1_n_caps, caps1_n_dims, caps2_n_caps, caps2_n_dims, routings=3):
        super(CapsuleLayer, self).__init__()
        self.caps1_n_caps = caps1_n_caps 
        self.caps1_n_dims = caps1_n_dims
        self.caps2_n_caps = caps2_n_caps
        self.caps2_n_dims = caps2_n_dims
        self.routings = routings
        self.W = torch.nn.Parameter(torch.normal(0, 0.1, size=(1, self.caps1_n_caps, self.caps2_n_caps, self.caps2_n_dims, self.caps1_n_dims)))
        self.ms=torch.nn.Softmax(2).cuda()
        
            
    def forward(self,x):
        #Las copias para poder multiplicar por la matriz de pesos
        batch_size=x.shape[0]
        x_expanded = x.unsqueeze(-1)    
        x_expanded_tile = x_expanded.unsqueeze(2)
        x_tiled = x_expanded_tile.repeat(1, 1, self.caps2_n_caps, 1,1)
        W_tile=self.W.repeat(batch_size,1,1,1,1)
        W_tile=W_tile.cuda()
        #Multiplicacion Capsulas W
        x_tiled = x_tiled.cuda()
        x_predicted = torch.matmul(W_tile, x_tiled)
        #x_hat_detached = x_predicted.detach()
        
        b = torch.zeros([batch_size, self.caps1_n_caps, self.caps2_n_caps, 1, 1])
        b =b.cuda()
        for i in range(self.routings):
            b_weights =self.ms(b)
            if i == self.routings - 1:
                outputs = squash(torch.sum(torch.mul(b_weights, x_predicted),1, keepdim=True))
            else:
                outputs = squash(torch.sum(torch.mul(b_weights, x_predicted),1, keepdim=True))                                 
                b = b + torch.matmul(torch.transpose(x_predicted,3,4), outputs.repeat(1, caps1_n_caps, 1, 1, 1))

        return(outputs)

In [ ]:
class PrimaryCapsule(torch.nn.Module):
    """
    Aplica Convolucion y transforma 
    :param in_channels: inputs conv
    :param out_channels: output conv
    :param caps1_n_dims: dimension de la capsula
    :param caps1_n_caps: numero de capsulas
    :param kernel_size: kernel size
    :return: output tensor, size=[batch, caps1_n_caps, caps1_n_dims]
    """
    def __init__(self, in_channels, out_channels, caps1_n_caps, caps1_n_dims, kernel_size=9, stride=2):
        super(PrimaryCapsule, self).__init__()
        self.caps1_n_dims = caps1_n_dims
        self.caps1_n_caps = caps1_n_caps
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride)

    def forward(self, x):
        out = self.conv2d(x)
        out = out.cuda()
        outputs = torch.reshape(out, (-1, self.caps1_n_caps, self.caps1_n_dims))
        return squash(outputs)

In [ ]:
class CapsuleNet(torch.nn.Module):
    
    
    
    def __init__(self,in_channels, out_channels, caps1_n_caps, caps1_n_dims,caps2_n_caps, caps2_n_dims, routings=3):
        super(CapsuleNet, self).__init__()
        self.conv=torch.nn.Conv2d(3, 256, (171,227), stride=(22,29))
        self.relu = torch.nn.ReLU()
        self.Primary=PrimaryCapsule(in_channels, out_channels, caps1_n_caps, caps1_n_dims)
        self.Capsule=CapsuleLayer(caps1_n_caps, caps1_n_dims, caps2_n_caps, caps2_n_dims, routings=3)
        
        
    def forward(self,x):
        out=self.relu(self.conv(x))
        out=self.Primary(out)
        out=self.Capsule(out)
        y_proba = safe_norm(out, axis=-2)
        y_proba_argmax = torch.argmax(y_proba, axis=2)
        y_pred = torch.squeeze(torch.squeeze(y_proba_argmax,2),1)
        return(y_pred,out)

In [ ]:
class CapsuleNet(pl.LightningModule):
#model=CapsuleNet(256, 256, caps1_n_caps, caps1_n_dims,caps2_n_caps, caps2_n_dims, routings=3)


    def __init__(self,in_channels=256, out_channels=256, caps1_n_caps=1152, caps1_n_dims=8,caps2_n_caps=4, caps2_n_dims=16, routings=3):
        super(CapsuleNet, self).__init__()
        self.conv=torch.nn.Conv2d(3, 256, (171,227), stride=(22,29))
        self.relu = torch.nn.ReLU()
        self.Primary=PrimaryCapsule(in_channels, out_channels, caps1_n_caps, caps1_n_dims)
        self.Capsule=CapsuleLayer(caps1_n_caps, caps1_n_dims, caps2_n_caps, caps2_n_dims, routings=3)

    def forward(self,x):
        out=self.relu(self.conv(x))
        out=self.Primary(out)
        out=self.Capsule(out)
        y_proba = safe_norm(out, axis=-2)
        y_proba_argmax = torch.argmax(y_proba, axis=2)
        y_pred = torch.squeeze(torch.squeeze(y_proba_argmax,2),1)
        return(y_pred,out)

    def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        #x = self.trans(x)
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        #loss = margin_loss(y,y_hat)
        acc = accuracy(y_hat, y)
        # Logging to TensorBoard by default
        self.log('loss',loss)
        self.log('acc', acc, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y)
        # Logging to TensorBoard by default
        self.log('val_loss',loss,prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        return optimizer

In [ ]:
config_2 = {'lr':3e-4,
          'batch_size':32,
          'precision':16,
          'epochs':3,
          'subset': 0.1,
          'size':256
         }

dm = DataModule(batch_size=config_2['batch_size'], subset=config_2['subset'])
ds=dm.train_ds

In [ ]:
config_2 = {'lr':3e-4,
          'batch_size':32,
          'precision':16,
          'epochs':3,
          'subset': 0.1,
          'size':256
         }

dm = DataModule(batch_size=config_2['batch_size'], subset=config_2['subset'])

model_2 = CapsuleNet(config_2)

trainer = pl.Trainer(gpus=1,
    precision=config_2['precision'],
    max_epochs=config_2['epochs'],
    limit_train_batches = 5,
    limit_val_batches = 5
    )


trainer.fit(model_2,dm)

# TERCER INTENTO

In [ ]:
ds = dm.train_ds
ix = random.randint(0, len(ds)-1)
img, lablel = ds[ix]
label = train['label'][ix]
#img=img.permute(1,2,0)#(3x600x800)
img.shape

In [ ]:
#r, c = 3, 5
#fig = plt.figure(figsize=(3*c, 3*r))
ds = dm.train_ds
ix = random.randint(0, len(ds)-1)
img, lablel = ds[ix]
label = train['label'][ix]
#img=img.permute(1,2,0)#(3x600x800)
img=torch.reshape(img, (1,3,600,800))
conv_1 = torch.nn.Conv2d(3, 5, (159,224),(9,9))
out1=conv_1(img)
conv_2 = torch.nn.Conv2d(5, 10, (15,30), (7,5))
out2 = conv_2(out1)
out3 = torch.reshape(out2, (-1, 20, 1))
out4 = squash(out3)
#out4.shape #24x20x1
u=out4[0,:,:]
W1 = torch.normal(0,0.01, size=(1, 24,5,16,10))
#W = torch.nn.Parameter(torch.normal(0, 0.1, size=(5,20)),requires_grad=True)
#W2 = torch.tile(W1, (2, 2))
u2 = torch.unsqueeze(out4, 2)
u3 = torch.tile(u2, (1,1,5,1,1))

torch.matmul(W1,u3)

In [ ]:

u2 = torch.unsqueeze(out4, 2)
u3 = torch.tile(u2, (1,1,5,1,1))

u2.shape

In [ ]:
class PrimaryCapsule(torch.nn.Module):

    def __init__(self):
        super(PrimaryCapsule, self).__init__()
        self.conv_1 = torch.nn.Conv2d(3, 5, (159,224),(9,9))#salida de un mapa de caracteristicas de 5*(50*65)
        self.conv_2 = torch.nn.Conv2d(5, 10, (15,30), (7,5))#imagen de salida de 10*(6*8)

    def forward(self, x):
        out = self.conv_1(x)
        out = self.conv_2(out)
        outputs = torch.reshape(out, (-1, 20, 1))#24*20*1, con la intencion de sacar 24  vectores de dimension 20
        return squash(outputs)

In [ ]:
class CapsuleLayer(torch.nn.Module):
    #Capa capsulada
    #caps1_n_caps : numero de capsulas de entrada
    #caps1_n_dims: dimension de las capsulas de entrada
    #caps2_n_caps: numero de capsulas de salida
    #caps2_n_dims: dimension de las capsulas de salida
    #routings: iteraciones del routing
    def __init__(self):#, caps1_n_caps, caps1_n_dims, caps2_n_caps, caps2_n_dims, routings=3):
        super(CapsuleLayer, self).__init__()
        #self.caps1_n_caps = caps1_n_caps 
        #self.caps1_n_dims = caps1_n_dims
        #self.caps2_n_caps = caps2_n_caps
        #self.caps2_n_dims = caps2_n_dims
        #self.routings = routings
        self.W = torch.nn.Parameter(torch.normal(0, 0.1, size=(5,20)),requires_grad=True)#pasar de 24 vect de dim 20 a 5 
        self.ms=torch.nn.Softmax(24)
        self.batch_size=32
        
            
    def forward(self,x):
        #Las copias para poder multiplicar por la matriz de pesos
        x_expanded = x.unsqueeze(-1)    
        x_expanded_tile = x_expanded.unsqueeze(2)
        x_tiled = x_expanded_tile.repeat(1, 1, self.caps2_n_caps, 1,1)
        W_tile=self.W.repeat(self.batch_size,1,1,1,1)
        #Multiplicacion Capsulas W
        x_tiled = x_tiled.cuda()
        x_predicted = torch.matmul(W_tile, x_tiled)
        #x_hat_detached = x_predicted.detach()
        
        b = torch.zeros([batch_size, self.caps1_n_caps, self.caps2_n_caps, 1, 1])
        b =b.cuda()
        for i in range(self.routings):
            b_weights =self.ms(b)
            if i == self.routings - 1:
                outputs = squash(torch.sum(torch.mul(b_weights, x_predicted),1, keepdim=True))
            else:
                outputs = squash(torch.sum(torch.mul(b_weights, x_predicted),1, keepdim=True))                                 
                b = b + torch.matmul(torch.transpose(x_predicted,3,4), outputs.repeat(1, caps1_n_caps, 1, 1, 1))

        return(outputs)